## Подбор параметров модели методом поиска на сетке - GridSearch 

Поиск оптимальных значений ключевых параметров модели (то есть значений, которые дают наилучшую обобщающую
способность) является сложной задачей, но она обязательна почти для всех моделей и наборов данных. Поскольку поиск оптимальных значений параметров является общераспространенной задачей, библиотека **scikit-learn** предлагает стандартные методы, позволяющие решить ее.

Наиболее часто используемый метод – это решетчатый поиск (**grid search**), который по сути является перебором всех возможных комбинаций интересующих параметров.


Рассмотрим модель дерева решений DecisionTreeClassifier на известном наборе данных и оценим качество классификации при различных значениях параметров модели:

* min_samples_leaf - минимальное кол-во примеров в листе (по умолчанию 1)
* min_samples_split - минимальное кол-во примеров для ветвления (по умолчанию 2)
* max_depth - максимальная глубина дерева (по умоланию не ограничено)

Значения по умолчанию часто приводят к заметному перебочунию дерева решений (на обучающей выборке 1.0, а на тестовой значительно ниже)

Покрутим эти параметры по сетке из следующих значений:

* min_samples_leaf = {1, 2, ..10}
* min_samples_split = {2,4, ... 10}
* max_depth = {2,3,...20}


Класс **GridSearchCV** выполняет подбор параметров указанной модели на заданной сетке значений с кросс-валидацией модели, поскольку хорошие результаты для набора параметров могли быть получены на случайно удачном наборе данных, потому для проверки помимо разбиения на train/test для каждой модели необходимо также выделить валидационную часть общего набора данных, либо выполнять кросс-валидацию, которую **GridSearchCV** и выполняет.

In [3]:
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier

iris = load_iris()

In [11]:
dtc = DecisionTreeClassifier()

params = { 'max_depth': range (2, 20, 1),
              'min_samples_leaf': range (1, 10),
              'min_samples_split': range (2,10,2) }

grid = GridSearchCV(dtc, params, cv = 5, n_jobs = -1)
grid.fit(iris.data, iris.target)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'max_depth': range(2, 20),
                         'min_samples_leaf': range(1, 10),
                         'min_samples_split': range(2, 10, 2)})

In [12]:
print(f"Наилучшие значения параметров: {grid.best_params_}")
print(f"Наилучшее значение метрики: {grid.best_score_}")

Наилучшие значения параметров: {'max_depth': 3, 'min_samples_leaf': 1, 'min_samples_split': 8}
Наилучшее значение метрики: 0.9733333333333334


<hr>

## Ансамбли моделей. Случайный лес (RandomForestClassifier)

В машинном обучении под ансамблем моделей понимают комбинацию нескольких алгоритмов обучения, которые, работая вместе, позволяют построить модель более эффективную и точную, чем любая из моделей, построенная с помощью отдельного алгоритма.

случайный лес – это набор деревьев решений, где каждое дерево немного отличается от остальных. Идея случайного леса заключается в том, что каждое дерево может довольно хорошо прогнозировать, но скорее всего переобучается на части данных. Если мы построим много деревьев, которые хорошо работают и переобучаются с разной степенью, мы можем уменьшить переобучение путем усреднения их результатов.

In [13]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, train_size = 0.75)

In [15]:
forest = RandomForestClassifier(n_estimators = 5)
forest.fit(X_train, y_train)

RandomForestClassifier(n_estimators=5)

In [18]:
print("Правильность на обучающем наборе: {:.3f}".format(forest.score(X_train, y_train)))
print("Правильность на тестовом наборе: {:.3f}".format(forest.score(X_test, y_test)))

Правильность на обучающем наборе: 0.991
Правильность на тестовом наборе: 0.974


Подберем наилучшие параметры для случайного леса методом **GridSearchCV**

In [19]:
params = {  "n_estimators": range(3,20),
            'max_depth': range (2, 20, 1),
            'min_samples_leaf': range (1, 10),
            'min_samples_split': range (2,10,2) }

gridRf = GridSearchCV(forest, params, cv = 5, n_jobs = -1)
gridRf.fit(iris.data, iris.target)

GridSearchCV(cv=5, estimator=RandomForestClassifier(n_estimators=5), n_jobs=-1,
             param_grid={'max_depth': range(2, 20),
                         'min_samples_leaf': range(1, 10),
                         'min_samples_split': range(2, 10, 2),
                         'n_estimators': range(3, 20)})

In [20]:
print(f"Наилучшие значения параметров: {gridRf.best_params_}")
print(f"Наилучшее значение метрики: {gridRf.best_score_}")

Наилучшие значения параметров: {'max_depth': 17, 'min_samples_leaf': 8, 'min_samples_split': 6, 'n_estimators': 8}
Наилучшее значение метрики: 0.9800000000000001
